In [1]:
import pandas as pd
import numpy as np
import requests
import json
from collections import Counter

In [2]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
def CountList(e):
    return Counter(e)
def Uniquefy(e):
    return list(set(e))

In [4]:
# Leitura do dataset
champ = pd.read_csv('participants.csv')
champ = champ.loc[champ['gameDuration'] > 1500]
champ['played'] = 1

C:\Users\zenet\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
gpby = champ.groupby(['participantId'])['championId'].apply(list).reset_index(name='championsPlayed')
gpby['count'] = gpby['championsPlayed'].apply(len)
gpby = gpby.loc[gpby['count'] > 5]

In [6]:
df_usable = gpby.explode('championsPlayed')
df_usable['played'] = 1
df_usable.drop('count',axis='columns', inplace=True)
df_usable

,participantId,championsPlayed,played
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,14,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,84,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,62,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,164,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,106,1
...,...,...,...
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,3,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,147,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,12,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,12,1


In [7]:
df_grouped = df_usable.groupby(['participantId', 'championsPlayed'])['played'].sum()
cesta = df_grouped.unstack(fill_value=0)
cesta.reset_index().set_index('participantId')
df_grouped

participantId                                             championsPlayed
--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqCsNQ3IshLwd  6                  4
                                                          14                 2
                                                          24                 2
                                                          31                 2
                                                          39                 3
                                                                            ..
zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA           117                1
                                                          147                5
                                                          201                1
                                                          235                2
                                                          526                3
Name: played, Length: 14118, dtype: int64

In [8]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

cesta = cesta.applymap(encode_units)
frequent_itemset = apriori(cesta, min_support=0.01, use_colnames=True)

In [12]:
def compare (a):
    return a.__contains__(497)
rules = association_rules(frequent_itemset,metric="confidence",min_threshold=0.01)
rules_test = rules[rules['antecedents'].map(compare)]
rules_test.sort_values('support', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
224,(497),(12),0.061343,0.127315,0.034722,0.566038,4.445969,0.026912,2.010970
2374,(497),(412),0.061343,0.144097,0.031250,0.509434,3.535349,0.022411,1.744725
826,(497),(43),0.061343,0.151620,0.030671,0.500000,3.297710,0.021371,1.696759
1816,(497),(111),0.061343,0.079282,0.026042,0.424528,5.354634,0.021178,1.599935
1625,(497),(89),0.061343,0.090856,0.025463,0.415094,4.568682,0.019890,1.554342
3814,"(497, 412)",(12),0.031250,0.127315,0.024884,0.796296,6.254545,0.020906,4.284091
3817,(497),"(412, 12)",0.061343,0.063657,0.024884,0.405660,6.372556,0.020979,1.575434
3815,"(497, 12)",(412),0.034722,0.144097,0.024884,0.716667,4.973494,0.019881,3.020833
1884,(497),(117),0.061343,0.125000,0.024306,0.396226,3.169811,0.016638,1.449219
2292,(497),(235),0.061343,0.146412,0.024306,0.396226,2.706242,0.015324,1.413755
